In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder


In [2]:
df = pd.read_csv('./Data/all.data.csv')
df.shape
df.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50k
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50k
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50k
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50k
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50k


In [3]:
df.drop(columns=["fnlwgt", "relationship"], inplace=True)
df.head()

,age,workclass,education,education-num,marital-status,occupation,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,White,Male,2174,0,40,United-States,<=50k
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,White,Male,0,0,13,United-States,<=50k
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,White,Male,0,0,40,United-States,<=50k
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Black,Male,0,0,40,United-States,<=50k
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Black,Female,0,0,40,Cuba,<=50k


In [4]:
encoders = [(["sex"], LabelEncoder()), (["race"], LabelEncoder())]
mapper = DataFrameMapper(encoders, df_out=True)
new_cols = mapper.fit_transform(df.copy())
df = pd.concat([df.drop(columns=["sex", "race"]), new_cols], axis="columns")
df.head()


C:\Users\Dumindu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Dumindu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,age,workclass,education,education-num,marital-status,occupation,capital-gain,capital-loss,hours-per-week,native-country,salary,sex,race
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,2174,0,40,United-States,<=50k,1,4
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,0,0,13,United-States,<=50k,1,4
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,0,0,40,United-States,<=50k,1,4
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,0,0,40,United-States,<=50k,1,2
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,0,0,40,Cuba,<=50k,0,2


In [7]:
from dist_utils import best_fit_distribution
from dist_utils import plot_result

categorical = ['race']
continuous = ['age']
unchanged = []

for col in list(df):
    if (col not in categorical) and (col not in continuous):
        unchanged.append(col)

best_distributions = []
for col in continuous:
    data = df[col]
    best_dist_name, best_dist_params = best_fit_distribution(data, 500)
    best_distributions.append((best_dist_name, best_dist_params))


def perturb_data(df, unchanged_cols, categorical_cols, continuous_cols, best_distributions, n, seed=0):
    np.random.seed(seed)
    data = {}

    for col in categorical_cols:
        counts = df[col].value_counts()
        data[col] = np.random.choice(list(counts.index), p=(counts/len(df)).values, size=n)

    for col, bd in zip(continuous_cols, best_distributions):
        dist = getattr(scipy.stats, bd[0])        
        data[col] = np.round(dist.rvs(size=n, *bd[1]))
        
        
    for col in unchanged_cols:
        data[col] = df[col]
   
    return pd.DataFrame(data, columns=unchanged_cols+categorical_cols+continuous_cols)    

gendf = perturb_data(df, unchanged, categorical, continuous, best_distributions, n=48842)
gendf.head() 

,workclass,education,education-num,marital-status,occupation,capital-gain,capital-loss,hours-per-week,native-country,salary,sex,race,age
0,State-gov,Bachelors,13,Never-married,Adm-clerical,2174,0,40,United-States,<=50k,1,4,65.0
1,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,0,0,13,United-States,<=50k,1,4,35.0
2,Private,HS-grad,9,Divorced,Handlers-cleaners,0,0,40,United-States,<=50k,1,4,80.0
3,Private,11th,7,Married-civ-spouse,Handlers-cleaners,0,0,40,United-States,<=50k,1,4,47.0
4,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,0,0,40,Cuba,<=50k,0,4,56.0


In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from cn.protect import Protect
sns.set(style="darkgrid")

df = pd.read_csv('./Data/all.data.csv')

df.shape
df.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50k
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50k
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50k
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50k
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50k


In [17]:
print('age\t\t    QUASI')
print('workclass\tINSENTIVE')
print('fnlwgt\t\tINSENTIVE')
print('education\tINSENTIVE')
print('education-num\tINSENTIVE')
print('marital-status\tINSENTIVE')
print('occupation\tINSENTIVE')
print('relationship\tINSENTIVE')
print('race\t\tINSENTIVE')
print('sex\t\t    QUASI')
print('capital-gain\tINSENTIVE')
print('capital-loss\tINSENTIVE')
print('hours-per-week\tINSENTIVE')
print('native-country\tINSENTIVE')
print('salary\t\tINSENTIVE')
print('dtype: object')

age		    QUASI
workclass	INSENTIVE
fnlwgt		INSENTIVE
education	INSENTIVE
education-num	INSENTIVE
marital-status	INSENTIVE
occupation	INSENTIVE
relationship	INSENTIVE
race		INSENTIVE
sex		    QUASI
capital-gain	INSENTIVE
capital-loss	INSENTIVE
hours-per-week	INSENTIVE
native-country	INSENTIVE
salary		INSENTIVE
dtype: object


In [18]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50k
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50k
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50k
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50k
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50k
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50k
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50k
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50k


In [23]:
df["age"] = df["age"].replace([39], '*')
df["age"] = '*'
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,*,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50k
1,*,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50k
2,*,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50k
3,*,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50k
4,*,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,*,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50k
48838,*,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50k
48839,*,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50k
48840,*,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50k
